# 8. Gemini API + Pydantic 구조화 출력

**학습 목표**: Gemini API로 Pydantic 스키마 기반의 구조화된 JSON 응답을 받습니다.

**사전 준비**: 
- `pip install google-genai pydantic python-dotenv`
- GOOGLE_API_KEY 환경변수 설정

---

## 문법 설명

### 1. Gemini API 사용

**정의**: Google의 Gemini API를 사용하여 AI 응답을 받습니다.

**설치 및 임포트**:
```python
from google import genai
from dotenv import load_dotenv
import os
```

**클라이언트 생성**:
```python
load_dotenv()
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
```

**API 호출 (Responses API)**:
```python
response = client.responses.create(
    model="gemini-2.0-flash-exp",
    input=[
        {"role": "system", "content": "시스템 프롬프트"},
        {"role": "user", "content": "사용자 입력"}
    ],
    max_output_tokens=2000,
)
content = response.output_text
```

**중요 사항**:
- `client.responses.create()` 사용 (최신 API)
- `client.chat.completions.create()` 사용 금지 (구버전)
- `temperature` 파라미터 없음 (Responses API에서 제거됨)
- 입력은 `input=[{role, content}]` 형식
- 출력은 `response.output_text` 사용

---

### 2. Pydantic 구조화 출력

**정의**: Pydantic 모델을 스키마로 사용하여 구조화된 JSON 응답을 받습니다.

**스키마 정의**:
```python
from pydantic import BaseModel, Field

class 응답모델(BaseModel):
    필드1: 타입 = Field(description="설명")
    필드2: 타입
```

**구조화 출력 요청**:
```python
response = client.responses.create(
    model="gemini-2.0-flash-exp",
    input=[{"role": "user", "content": "프롬프트"}],
    response_schema=응답모델,  # Pydantic 모델 전달
    max_output_tokens=2000,
)
```

**응답 파싱**:
```python
result = response.output_text  # JSON 문자열
parsed = 응답모델.model_validate_json(result)  # Pydantic 객체로 변환
```

**장점**:
- 타입 안정성 보장
- 자동 검증
- IDE 자동완성 지원
- 문서화 용이

---
## 실습 시작

아래 실습을 통해 위 문법들을 직접 사용해봅니다.

---

In [1]:
#pip install google-genai pydantic python-dotenv

## 8.1 API 키 설정

In [2]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# API 키 확인
api_key = os.environ.get("GOOGLE_API_KEY")
if api_key:
    print(f"API 키 로드됨: {api_key[:8]}...")
else:
    print("API 키가 설정되지 않았습니다.")
    print("1. https://aistudio.google.com/apikey 에서 API 키 발급")
    print("2. .env 파일에 GOOGLE_API_KEY=your-api-key 추가")

API 키 로드됨: AIzaSyBA...


In [3]:
from google import genai

# 클라이언트 생성 (GOOGLE_API_KEY 환경변수 자동 사용)
client = genai.Client()

---
## 8.2 기본 텍스트 생성

In [4]:
# 간단한 텍스트 생성
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="파이썬의 장점 3가지를 간단히 알려주세요."
)

print(response.text)

파이썬의 장점 3가지는 다음과 같습니다.

1.  **배우기 쉽고 읽기 쉬운 문법:** 영어를 닮은 간결하고 직관적인 문법 덕분에 프로그래밍 초보자도 쉽게 배우고 코드를 이해하기 쉽습니다.
2.  **폭넓은 활용 분야와 방대한 라이브러리:** 웹 개발, 데이터 과학, 인공지능, 자동화 등 다양한 분야에서 활용되며, 이를 돕는 수많은 라이브러리(NumPy, Pandas, Django, TensorFlow 등)를 갖추고 있습니다.
3.  **높은 생산성과 활발한 커뮤니티:** 적은 코드로도 복잡한 기능을 구현할 수 있어 개발 시간을 단축시키고, 문제가 생겼을 때 도움을 받을 수 있는 거대한 사용자 커뮤니티가 있습니다.


---
## 8.3 Pydantic 스키마로 구조화 출력

핵심 포인트:
- `response_mime_type`: "application/json" 으로 설정
- `response_schema`: Pydantic 모델 클래스 전달
- `model_validate_json()`: 응답을 Pydantic으로 검증

In [5]:
from pydantic import BaseModel, Field
from typing import Literal, Optional

### 8.3.1 감성 분석

In [6]:
class SentimentResult(BaseModel):
    """감성 분석 결과"""
    sentiment: Literal["긍정", "부정", "중립"] = Field(description="감성 분류")
    confidence: float = Field(ge=0, le=1, description="신뢰도 0-1")
    keywords: list[str] = Field(default=[], description="핵심 키워드")
    summary: str = Field(description="한 줄 요약")

In [7]:
text = "이 제품은 정말 훌륭해요! 품질도 좋고 배송도 빨랐습니다. 다만 포장이 조금 아쉬웠어요."

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"다음 텍스트의 감성을 분석하세요:\n{text}",
    config={
        "response_mime_type": "application/json",
        "response_schema": SentimentResult,
    },
)

# Pydantic으로 검증
result = SentimentResult.model_validate_json(response.text)

print(f"감성: {result.sentiment}")
print(f"신뢰도: {result.confidence:.2f}")
print(f"키워드: {result.keywords}")
print(f"요약: {result.summary}")

감성: 긍정
신뢰도: 0.85
키워드: ['제품', '품질', '배송', '포장']
요약: 제품의 품질과 빠른 배송은 훌륭했지만 포장은 약간 아쉬웠습니다.


In [8]:
# model_dump()로 딕셔너리 변환
import json
print(json.dumps(result.model_dump(), ensure_ascii=False, indent=2))

{
  "sentiment": "긍정",
  "confidence": 0.85,
  "keywords": [
    "제품",
    "품질",
    "배송",
    "포장"
  ],
  "summary": "제품의 품질과 빠른 배송은 훌륭했지만 포장은 약간 아쉬웠습니다."
}


### 8.3.2 레시피 추출 (중첩 모델)

In [9]:
class Ingredient(BaseModel):
    """재료"""
    name: str = Field(description="재료 이름")
    quantity: str = Field(description="수량 (단위 포함)")

class Recipe(BaseModel):
    """레시피"""
    recipe_name: str = Field(description="요리 이름")
    prep_time_minutes: Optional[int] = Field(description="준비 시간 (분)")
    ingredients: list[Ingredient] = Field(description="재료 목록")
    instructions: list[str] = Field(description="조리 순서")

In [10]:
recipe_text = """
간단 계란볶음밥 만들기
재료: 밥 1공기, 계란 2개, 파 1줄기, 간장 1스푼, 참기름 약간
1. 팬에 기름을 두르고 파를 볶습니다.
2. 풀어둔 계란을 넣고 스크램블합니다.
3. 밥을 넣고 간장으로 간을 합니다.
4. 참기름을 뿌려 마무리합니다.
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"다음 텍스트에서 레시피를 추출하세요:\n{recipe_text}",
    config={
        "response_mime_type": "application/json",
        "response_schema": Recipe,
    },
)

recipe = Recipe.model_validate_json(response.text)

print(f"요리: {recipe.recipe_name}")
print(f"준비 시간: {recipe.prep_time_minutes}분")
print(f"\n재료:")
for ing in recipe.ingredients:
    print(f"  - {ing.name}: {ing.quantity}")
print(f"\n조리 순서:")
for i, step in enumerate(recipe.instructions, 1):
    print(f"  {i}. {step}")

요리: 간단 계란볶음밥
준비 시간: None분

재료:
  - 밥: 1공기
  - 계란: 2개
  - 파: 1줄기
  - 간장: 1스푼
  - 참기름: 약간

조리 순서:
  1. 팬에 기름을 두르고 파를 볶습니다.
  2. 풀어둔 계란을 넣고 스크램블합니다.
  3. 밥을 넣고 간장으로 간을 합니다.
  4. 참기름을 뿌려 마무리합니다.


---
## 연습문제

### 회의록 요약 모델
회의 내용을 요약하는 Pydantic 모델을 만들고 Gemini로 요약하세요.
- participants: 참가자 리스트
- main_topics: 주요 안건
- decisions: 결정 사항
- next_steps: 다음 단계

`data/08_회의록.txt`의 sample 회의록 사용

In [11]:
class MeetingSummary(BaseModel):
    # 여기에 코드 작성
    pass